In [ ]:
# 0) Load your flattened dataset
import pandas as pd

df_flat = pd.read_csv("data/beyondblue_data_flattened.csv")
df_flat['text'] = df_flat['text'].astype(str).fillna("")
df_flat.head()

In [1]:
# 2) force Transformers to use PyTorch only (avoid TF/Keras path)
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["USE_TF"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# 3) verify versions and device
import torch, transformers
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("CUDA available?", torch.cuda.is_available())
DEVICE = 0 if torch.cuda.is_available() else -1

torch: 2.3.1
transformers: 4.41.2
CUDA available? False


In [3]:
# sentiment pipeline (no change needed, just ignore pooler warning)
from transformers import pipeline

sentiment_pipe = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    framework="pt",
    device=DEVICE
)

# emotion pipeline (updated for new API)
emotion_pipe = pipeline(
    task="text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    framework="pt",
    top_k=None,    # replaces return_all_scores=True
    device=DEVICE
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 5) quick smoke test
texts = [
    "I can't sleep and my mind won't stop racing.",
    "Thank you for being there — I feel hopeful."
]
print("sentiment:", sentiment_pipe(texts))
print("emotion dist (first):", emotion_pipe(texts)[0])

sentiment: [{'label': 'negative', 'score': 0.862205445766449}, {'label': 'positive', 'score': 0.9792223572731018}]
emotion dist (first): [{'label': 'sadness', 'score': 0.8041720986366272}, {'label': 'neutral', 'score': 0.07814598083496094}, {'label': 'fear', 'score': 0.050828780978918076}, {'label': 'anger', 'score': 0.047112248837947845}, {'label': 'disgust', 'score': 0.008193270303308964}, {'label': 'surprise', 'score': 0.007894475944340229}, {'label': 'joy', 'score': 0.0036531074438244104}]
